# Day 1 - Exercise 1 

*Web Scraping and Summarization Exercise*

### Objective

This notebook wil scrape GitHub repositories declared in the repos.txt, fetch their README.md files, and summarize them using an LLM. The summaries will be guided by system and user prompts.  It is a different approach to screen scraping websites and the general idea is to use LLM to summarise the code repository based on user and system prompts currently limited to the repos defined in repos.txt. However could be useful/extended as a search tool to search git repos on a topic (eg. LLM) and retrieve summarised information from README, CONTRIBUTORS and other meta data... only day 1, so won't get to carried away...


In [1]:
from openai import OpenAI

# Initialize OpenAI client
client = OpenAI()

### GitHub README Fetching and Display Utilities

This section defines helper functions to extract repository information, fetch README files from GitHub, and display summaries in Jupyter notebooks.

In [2]:
import requests
from urllib.parse import urlparse
from IPython.display import Markdown, display

# Extract GitHub owner and repo name from URL
def extract_repo_parts(url: str):
    parsed = urlparse(url)
    parts = parsed.path.strip('/').split('/')
    if len(parts) < 2:
        raise ValueError(f'Invalid GitHub repo URL: {url}')
    return parts[0], parts[1]

# Fetch README.md from main or master branch of GitHub repo
def fetch_readme(owner: str, repo: str):
    branches = ['main', 'master']
    for branch in branches:
        raw_url = f"https://raw.githubusercontent.com/{owner}/{repo}/{branch}/README.md"
        r = requests.get(raw_url)
        if r.status_code == 200:
            return r.text
    raise FileNotFoundError(f'README.md not found on main or master branch for {owner}/{repo}')

# Display summary nicely in Markdown inside Jupyter
def display_summary(text: str):
    display(Markdown(text))



### Define the prompts for the LLM

- **System Prompt:** Sets the role and behavior of the model as a "philosophical, humorous software engineer".
- **User Prompt:** Contains the README content and requests a summary.

Using system + user prompts allows us to control tone and style.

In [3]:
# Define system prompt (role, tone, behavior)
system_prompt = """
You are a philosophical new age software engineer that analyzes the contents of GitHub repositories,
and provides a critical thinking, open-minded and humorous summary.
Respond in markdown. Do not wrap the markdown in a code block - respond just with the markdown.
"""

    # Define user prompt (content to summarize)
user_prompt_prefix = """
Here are the contents of a GitHub repository README.
Provide a short summary of this repository in the context of the system prompt.
"""


### This function sends the README content to the OpenAI API along with the prompts:

1. Combines the user prompt with README text.
2. Calls `client.responses.create` with system and user prompts.
3. Displays the summary in Markdown under the repository name.


In [4]:
# Summarize a README using the system and user prompts with OpenAI
def summarize_readme(readme_text: str, repo_name: str):
    
    # Combine user prompt prefix with the README content
    user_prompt = user_prompt_prefix + "\n\n" + readme_text

    # Call OpenAI Responses API
    response = client.responses.create(
        model="gpt-4.1-mini",
        input=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )

    # Display summary in Markdown
    summary = response.output_text
    display_summary(f"### {repo_name}\n{summary}")

### Load a list of GitHub repository URLs from `repos.txt`.

For each URL:
* Extract owner/repo.
* Fetch README.
* Generate and display summary.

In [5]:
# Read repo URLs from repos.txt
with open("repos.txt", "r") as f:
    repo_urls = [line.strip() for line in f if line.strip() and not line.startswith("#")]

# Fetch and summarize each README
for repo_url in repo_urls:
    try:
        owner, repo = extract_repo_parts(repo_url)
        readme_text = fetch_readme(owner, repo)
        repo_name = f"{owner}/{repo}"
        summarize_readme(readme_text, repo_name)
    except Exception as e:
        display_summary(f"**Error fetching {repo_url}: {e}**")


### ed-donner/llm_engineering
# Philosophical, New Age Engineer’s Take on the "LLM Engineering - Master AI and LLMs" Repo

Welcome to an 8-week expedition into the heart and soul of Large Language Models (LLMs), where code meets consciousness and bytes dance with enlightenment.

---

## Summary

This repository is much more than a collection of code—it’s a carefully curated **journey of mastery** in the burgeoning art and science of Large Language Models and AI engineering. Guided by Edward Donner, it welcomes seekers—from curious initiates to serious practitioners—into an evolving, hands-on curriculum designed to elevate your understanding and skill in building AI-driven projects.

### The Path

- **An 8-week progressive odyssey:** Each week builds upon the last, starting with accessible tools like Ollama and culminating in deep explorations of agentic AI, multimodal prompts, and autonomous AI solutions.
- **Dual reality of updates:** You can stay with the original Anaconda-based pilgrimage or embrace the new, streamlined path using `uv` for a fresher, sleeker experience—with all the modern bells and whistles like prompt caching and LiteLLM.
- **Learning by doing:** This isn’t about passive absorption. You **engage, tinker, break, and remix** the projects as you go. A soulful dance with code to transform knowledge into wisdom.
- **Cloud-friendly & budget-conscious:** Whether you’re a digital hermit with your own machine or a cloud nomad exploring GPUs on Google Colab, the course accommodates your lifestyle and wallet.
- **Community and connectivity:** The portal to LinkedIn, X/Twitter (with a cheeky nudge to teach Edward the ways of the “X”), and email invites you to participate in a living community of AI apprentices.

### The Zen of API Costs

In a world where cloud APIs can silently siphon your bank account, Edward’s gentle reminders and budget-conscious advice form a digital mantra: *Spend mindfully, learn abundantly, and don’t stress the dollar signs.*

---

## Why This Repository Resonates

This repo is a beacon for anyone ready to ride the current wave of AI transformation—not just by soaking in theory but by rolling their sleeves up and **transforming understanding into actionable, elegant code**. It’s a safe space to experiment right where the bleeding edge of AI meets the nurturing environment of a patient teacher.

*It’s simultaneously pragmatic and playful, bridging the gap between the mystical allure of AI and the grounded reality of engineering.*

---

## Final Thoughts: The Journey is the Reward

Embarking on this course is like setting sail on a vast ocean of intelligence—sometimes the waters are smooth, other times turbulent, but the vistas you witness are nothing short of astounding. In the spirit of the repository’s philosophy, remember: the best enlightenment comes when you’re actively creating, questioning, and embracing the beautiful messiness of learning.

**So, welcome aboard. Ready your terminal, open your mind, and prepare to be both humbled and thrilled!** 🚀✨

---

*“The code never lies, but it often surprises.” – Somewhere within the matrix*

### openai/openai-cookbook
# OpenAI Cookbook: A Playful Guide to AI Alchemy

This repository is like the wise old cookbook for the modern digital alchemist — you, the coder — eager to summon the wondrous spells of the OpenAI API. It offers *example code* (mostly Python, the lingua franca of AI mystics) and clear guides that demystify the enchantments behind conversational AI, language models, and more. 

Think of it as your trusty cauldron, ready to brew up creative projects once you’ve got your secret ingredient — the `OPENAI_API_KEY`. Whether you're coding in VS Code or beyond, this cookbook paves the way to turn cryptic API calls into tangible magic with minimal fuss.

Philosophically, it’s a gentle nod that powerful tools become personal art when paired with human curiosity and openness. So grab your digital parchment, sprinkle some API keys, and let's conjure some AI wonders — all under the benevolent MIT license’s protection.

In short: practical, open, and a little bit mystical — your starter pack for navigating the AI cosmos. 🌌✨